In [1]:
from deltakit.circuit.gates import PauliBasis, SQRT_X, SQRT_X_DAG, S_DAG, S, X, CZ, RZ, MZ, Z, H
from deltakit.explorer.codes import RotatedPlanarCode, UnrotatedPlanarCode, css_code_memory_circuit
from deltakit.explorer.qpu import QPU, SI1000Noise, NativeGateSet
from deltakit.decode.analysis import StimDecoderManager
from deltakit.decode import PyMatchingDecoder

# Run a QEC experiment with Deltakit and compare different decoders

In this notebook, you will run a memory experiment with Deltakit, and compare the logical error rates of different decoders to one another.

<img src="./QEC_experiment.png" width=500>

In [2]:
# Create code for desired distance: dd Rotated planar code
distance = 3
num_rounds = 3
code = RotatedPlanarCode(width=distance, height=distance)

In [3]:
# Create noiseless circuit: d-rounds of syndrome extraction, initialise in Z basis
noiseless_circuit = css_code_memory_circuit(
    code, num_rounds=num_rounds, logical_basis=PauliBasis.Z
)

In [4]:
# Print generated Stim circuit
print(noiseless_circuit.as_stim_circuit())

QUBIT_COORDS(2, 4) 0
QUBIT_COORDS(4, 0) 1
QUBIT_COORDS(1, 5) 2
QUBIT_COORDS(3, 1) 3
QUBIT_COORDS(5, 1) 4
QUBIT_COORDS(6, 4) 5
QUBIT_COORDS(0, 2) 6
QUBIT_COORDS(2, 2) 7
QUBIT_COORDS(1, 3) 8
QUBIT_COORDS(3, 5) 9
QUBIT_COORDS(4, 4) 10
QUBIT_COORDS(5, 5) 11
QUBIT_COORDS(1, 1) 12
QUBIT_COORDS(4, 2) 13
QUBIT_COORDS(3, 3) 14
QUBIT_COORDS(2, 6) 15
QUBIT_COORDS(5, 3) 16
R 8 16 14 9 4 12 11 2 3
RX 5 1 0 13 6 10 15 7
TICK
I 12 8 2 3 14 9 4 16 11
TICK
CX 5 11 0 2 13 14
CZ 1 3 10 9 7 8
TICK
CX 5 16 0 8 13 3
CZ 1 4 10 11 7 14
TICK
CX 0 9 13 16 6 8
CZ 10 14 15 2 7 12
TICK
CX 0 14 13 4 6 12
CZ 10 16 15 9 7 3
TICK
MX 5 1 0 13 6 10 15 7
TICK
DETECTOR(4, 0, 0) rec[-7]
DETECTOR(4, 4, 0) rec[-3]
DETECTOR(2, 6, 0) rec[-2]
DETECTOR(2, 2, 0) rec[-1]
SHIFT_COORDS(0, 0, 1)
RX 5 1 0 13 6 10 15 7
I 12 8 2 3 14 9 4 16 11
TICK
CX 5 11 0 2 13 14
CZ 1 3 10 9 7 8
TICK
CX 5 16 0 8 13 3
CZ 1 4 10 11 7 14
TICK
CX 0 9 13 16 6 8
CZ 10 14 15 2 7 12
TICK
CX 0 14 13 4 6 12
CZ 10 16 15 9 7 3
TICK
MX 5 1 0 13 6 10 15 7
TICK
DET

In [5]:
# Superconducting-inspired noise model at p=10^-3
noise_model = SI1000Noise(p=1e-3)

In [6]:
# QPU with standard native gate set
qpu = QPU(
    code.qubits,
    # native gates for superconducting qubits
    native_gates_and_times=NativeGateSet(
        one_qubit_gates={SQRT_X, SQRT_X_DAG, S_DAG, S, X, Z},
        two_qubit_gates={CZ},
        reset_gates={RZ},
        measurement_gates={MZ},
    ),
    noise_model=noise_model,
)

In [7]:
# Compile noiseless circuit to QU and add noise
noisy_circ = qpu.compile_and_add_noise_to_circuit(noiseless_circuit)

In [8]:
# Create a decoder
decoder, stim_circuit = PyMatchingDecoder.construct_decoder_and_stim_circuit(
    noisy_circ
)
# Run a simple experiment with 1e5 shots
num_shots = 1e5
experiment_decoder_manager = StimDecoderManager(
    stim_circuit, decoder
)
pct1_shots, pct1_fails = experiment_decoder_manager.run_batch_shots(num_shots)
print(f"Logical error rate for MWPW: {pct1_fails/pct1_shots}")

Logical error rate for MWPW: 0.00409


# Simulate and decode with Deltakit Cloud decoders

In addition to open source decoders, Deltakit supports Cloud-based decoders that you can access via the platform. You'll log in via https://deltakit.riverlane.com/ and get a token that will let you access the service.

In [9]:
from deltakit.explorer import Client
from deltakit.explorer.types import Decoder
from deltakit.explorer.enums import DecoderType

<img src="./cloud.png" width=350>

In [10]:
# # (Optional) Save token to env variable
# !echo "DELTAKIT_TOKEN=<token>" > ~/.config/deltakit-explorer/.env

In [11]:
# Create a client instance to connect
# Go to https://deltakit.riverlane.com/dashboard/token to copy your token
Client.set_token("<paste your token here>")

In [12]:
# Simulate circuit and get detectors and observables
client = Client.get_instance()
circuit = str(noisy_circ.as_stim_circuit())
measurements, _ = client.simulate_stim_circuit(circuit, int(1e5))
detectors, observables = measurements.to_detectors_and_observables(circuit)

In [13]:
# Decode results, loop over all available cloud decoders
results = []
decoders = DecoderType._member_map_.items()
for decoder_name, decoder_type in decoders:
    result = client.decode(
        detectors=detectors,
        observables=observables,
        decoder=Decoder(decoder_type),
        noisy_stim_circuit=circuit
    )
    results.append(result)

    # Print logical error rate (decoder fails/number of shots)
    print(
        f"Logical error rate for {decoder_name}: "
        f"{result.get_logical_error_probability()} in {sum(result.times):.5f} s"
    )

Logical error rate for MWPM: 0.00396 in 1.96775 s
Logical error rate for CC: 0.00485 in 1.10658 s
Logical error rate for BELIEF_MATCHING: 0.00331 in 3.58778 s
Logical error rate for BP_OSD: 0.00396 in 12.16231 s
Logical error rate for AC: 0.00732 in 6.77652 s
Logical error rate for LCD: 0.00408 in 1.43936 s


# Simulate and decode with Sinter using Fusion Blossom, MWPF, Relay-BP and Beliefmatching

You can now take the Stim circuits you've created using Deltakit to other tools like `sinter` and decoders like `relay_bp`.
Here is an example of using all decoders accessible via `sinter` and using them to decode the Stim circuit.

In [14]:
# !pip install sinter fusion_blossom MWPF~=0.1.5 relay_bp beliefmatching

In [15]:
import sinter
from relay_bp.stim import sinter_decoders

num_workers = 1
shots = int(1e5)
circuit = noisy_circ.as_stim_circuit()

decoders = sinter_decoders(
    gamma0=0.1,
    pre_iter=80,
    num_sets=60,
    set_max_iter=60,
    gamma_dist_interval=(-0.24, 0.66),
    stop_nconv=1,
)

all_decoders = list(sinter.BUILT_IN_DECODERS.keys()) + list(decoders.keys())
tasks = []
for decoder in all_decoders:
    task = sinter.Task(
        circuit=circuit,
        decoder=decoder,
        collection_options=sinter.CollectionOptions(max_shots=shots)
    )
    tasks.append(task)
    
# Collect the samples
samples = sinter.collect(
    tasks=tasks,
    num_workers=num_workers,
    custom_decoders=decoders,
)

for decoder, task_output in zip(all_decoders, samples):
    print(f"Logical error rate for {decoder}: {task_output.errors/task_output.shots}")

Logical error rate for vacuous: 0.05004
Logical error rate for pymatching: 0.00381
Logical error rate for fusion_blossom: 0.00422
Logical error rate for hypergraph_union_find: 0.0033
Logical error rate for mw_parity_factor: 0.00266
Logical error rate for relay-bp: 0.00854
Logical error rate for mem-bp: 0.0039
Logical error rate for msl-bp: 0.00429
